In [ ]:
# AGREGADO

# seleccione pip y CUDA 11.7 (ver si funciona para ti)
# https://pytorch.org/
!pip3 install torch torchvision torchaudio

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# AGREGADO

# https://pypi.org/project/opencv-python/
!pip install opencv-python

# https://github.com/open-mmlab/mmsegmentation/pull/1568/commits/3cdac2f1cb274dc9fc8c9530f997c7ece9ffe074
!apt-get update && apt-get install -y git ninja-build libglib2.0-0 libsm6 libxrender-dev libxext6 libgl1-mesa-glx \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*

In [ ]:
#MODIFICADO 

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# MODIFICADO 

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/tf/josue/filtro_3/best_weights/mymodel_17-0.0759')

# Check its architecture
model.summary()

In [ ]:
# AGREGADO 

ruta_imagenes = "/tf/josue/filtro_unificado/im_abarth_500_2012"
ruta_f1       = "/tf/josue/filtro_unificado/im_f1_no_carro"
ruta_f2       = "/tf/josue/filtro_unificado/im_f2_carro_pequeno"
ruta_f3       = "/tf/josue/filtro_unificado/im_f3_rueda_o_interior"
ruta_r        = "/tf/josue/filtro_unificado/im_fr_resultado"

def filtros_concatenados(ruta_imagenes, ruta_f1, ruta_f2, ruta_f3, ruta_r):
  mi_ruta = ruta_imagenes
  for count,a in enumerate(os.listdir(mi_ruta)):
    #print(count)
    try:
          
      # filtro 1 (al menos un carro)
      im = cv2.imread(mi_ruta +"/"+a)
      outputs = predictor(im)
      if (2 in outputs["instances"].pred_classes):
          
        # filtro 2 (el mayor objeto debe ser carro)
        objects_area = outputs["instances"].pred_boxes.area()
        objects_coor = outputs["instances"].pred_boxes.tensor
        objects_area_numpy = objects_area.cpu().numpy()
        objects_coor_numpy = objects_coor.cpu().numpy()
        max_index = np.argmax(objects_area_numpy)
        if 2 == int(outputs["instances"].pred_classes[max_index]):        
          
          # filtro 3 (solo carros desde el exterior)
          b = cv2.resize(im,(160,160))
          img_array = np.expand_dims(b, axis=0)
          class_names = ["interior","exterior","llanta"]
          predictions = model.predict(img_array)
          score = tf.nn.softmax(predictions[0])
          if (int(np.argmax(score)) == 1):
                    
            # filtro 4 (recorte)
            eee = objects_coor_numpy.tolist()[max_index]
            os.chdir(ruta_r)
            cv2.imwrite(a, im[int(eee[1]):int(eee[3]), int(eee[0]):int(eee[2])])
            
          else:
            os.chdir(ruta_f3)
            cv2.imwrite(a, im)
        
        else:
          os.chdir(ruta_f2)
          cv2.imwrite(a, im)
        
      else:
        os.chdir(ruta_f1)
        cv2.imwrite(a, im)
      
    except Exception as e: 
      print(e)


filtros_concatenados(ruta_imagenes, ruta_f1, ruta_f2, ruta_f3, ruta_r)

In [ ]:
'''
observaciones:

##### experimento 1 #####

- exterior
147_4_d9820971-a93c-362c-e053-e350040a1ccf_14d46714-a3c1-4e2f-b591-f4e13a027bfc.jpg
147_6_d9820971-a93c-362c-e053-e350040a1ccf_881025c1-04e5-4d7f-98e9-0decb03a2414.jpg

- rueda
151_12_65cfb222-f101-4bb7-96bc-b5312ff7e051_5bb73905-8bb8-404f-8883-737794ca2cba.jpg

- basura
152_11_65a74b92-a3a2-416e-bbe2-c37b60c41ba6_4d699502-a2c7-4050-9db9-e65254e5dccd.jpg

- podria recortarse
147_2_d9820971-a93c-362c-e053-e350040a1ccf_bfbbcede-5d10-4ad1-8fb3-b2e1e9be352d.jpg


##### experimento 2 #####

- es moto
152_11_65a74b92-a3a2-416e-bbe2-c37b60c41ba6_4d699502-a2c7-4050-9db9-e65254e5dccd.jpg

- es rueda
144_15_f1b7c739-cef4-4dbb-b271-44ed28f41267_417704fd-13e6-4baa-a3c6-0033b3e4923c.jpg
149_9_4ca3a46b-77e2-f215-e053-e350040ac2d0_2c1e4b32-9c71-47c4-8735-5993e2d65a97.jpg
128_4_a20c7742-c058-4690-b2ec-cd8a9e9fb499_dfe9fd10-e8e0-49a3-9251-01c0e23da67d.jpg

- es interior
147_6_d9820971-a93c-362c-e053-e350040a1ccf_881025c1-04e5-4d7f-98e9-0decb03a2414.jpg


##### experimento 3 #####

- ex exterior
136_4_67433e68-0a47-9c23-e053-e250040ab22d_1cc907a7-339c-4a71-8cd1-b5a8c5af401d.jpg
136_8_67433e68-0a47-9c23-e053-e250040ab22d_477ecb7d-76cb-4a76-92e6-7f3ac0572ae0.jpg

- parte de carro... deberia de eliminarse?
149_5_4ca3a46b-77e2-f215-e053-e350040ac2d0_ad191c7b-ab06-4e36-9722-bf6b9a70b535.jpg

- paso el filtro 1 pero no el 2. En teoria hay una box mas grande que no es carro pero al usar el script de detectron2 solo vemos un carro. Revisar.
138_11_4a61a874-0b3e-2055-e053-e250040a8ba3_2576dc6f-ddbf-44a0-89bd-d9c219cb3b0e.jpg
'''